# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

Please make sure if you are running this notebook in the workspace that you have chosen GPU rather than CPU mode.

In [2]:
# Imports here
from time import time, sleep # or should it be from time import time, sleep?
import numpy as np
import os

# from torchvision import transforms to change images
import torchvision.transforms as transforms

# from torchvision import models to call pretrained models
import torchvision.models as models

# datasets and DataLoader for image loading
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

# import json for label to image matching
import json

# import torch for new classifier
import torch
import torch.nn as nn

# import optim to update the weights with gradients
import torch.optim as optim

import json

# import module used to maintain an active session 
# during long-running processes
from workspace_utils import active_session

import argparse

from PIL import Image 

from matplotlib import pyplot as plt

# TODO  - figure out why workspace_utils isn't working
# from workspace_utils import active_session
# context manager can be used to keep session 
# active during long running process, see line below
# with active_session():
    # do long-running work here

# address of data  
# https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz

# Resources accessed in preparing this project:
# 7/14/2020: https://pytorch.org/docs/stable/torchvision/models.html
# 7/14/2020: https://pillow.readthedocs.io/en/latest/reference/Image.html

# 7/15/2020: https://www.programcreek.com/python/example/104834/torchvision.transforms.Resize
# 7/15/2020: https://www.programcreek.com/python/example/104835/torchvision.transforms.CenterCrop

# 7/16/2020: https://towardsdatascience.com/a-beginners-tutorial-on-building-an-ai-image-classifier-using-pytorch-6f85cb69cba7
# 7/16/2020: https://pytorch.org/docs/stable/data.html#data-loading-order-and-sampler

# 7/17/2020: https://pytorch.org/docs/master/generated/torch.nn.CrossEntropyLoss.html
# 7/17/2020: https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html#finetuning-the-convnet
# 7/17/2020: https://medium.com/@Biboswan98/optim-adam-vs-optim-sgd-lets-dive-in-8dbf1890fbdc






In [4]:
# flatten image? should be somewhere in the beginning
        # Flatten image sets images.shape to batch size 
        # and -1 tells it to fill in enough to meet needs of file
        #images = images.view(images.shape[0], -1)

#  prob   assign correct number of items to vgg list

# The two matrices m1 and m2 are getting multiplied and for matrix multiplication the column of first matrix and row of second matrix needs to be same. Hence size mismatch.


# ----> 8 optimizer = optim.SGD(model.parameters(), lr=learning_rate ),   trying adam

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [5]:
# dataset split into three parts
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [6]:
# transforms.Compose applied to 3 datasets

# TODO? resize/crop     both verticle/horizontal flip

# training data resized, randomly cropped, 
# randomly flipped, normalized mean and std
# ToTensor converts image into numbers
train_transforms = transforms.Compose([
    transforms.RandomRotation(27),
    transforms.Resize(255),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

# validation data resized with normalized mean and std
# ToTensor converts image into numbers
valid_transforms = transforms.Compose([
    transforms.Resize(255),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

# test data resized with normalized mean and std
# ToTensor converts image into numbers
test_transforms = transforms.Compose([
    transforms.Resize(255),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

# Load the datasets with ImageFolder and apply transforms
train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
valid_dataset = datasets.ImageFolder(valid_dir, transform=valid_transforms)
test_dataset = datasets.ImageFolder(test_dir, transform=test_transforms)

# load images using dataloder from torch library
# batching allows dataloader automatically fetch data samples
# shuffle=True the data gets “shuffled” before each epoc

train_loaders = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loaders = DataLoader(valid_dataset, batch_size=16, shuffle=True)
test_loaders = DataLoader(test_dataset, batch_size=16, shuffle=True)

# TODO: ?? used batch_size of 2 since lower seems to 
# produce more random results. Does the class need to be divisible
# by the batch_size?  Not sure the role of batch_size,
# but read that it the number of classes need to be divisible by batch_size. 32?

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [7]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)
    

In [8]:
# Checks to verify current status
# verifying items in list, # of images in 3 groups

print(len(cat_to_name)) 
print(cat_to_name)
print(len(train_loaders))
print(len(valid_loaders))
print(len(test_loaders))

# test if active session works during processes
with active_session():
    # do long-running work here
    sleep(4)
print('active session works')

102
{'21': 'fire lily', '3': 'canterbury bells', '45': 'bolero deep blue', '1': 'pink primrose', '34': 'mexican aster', '27': 'prince of wales feathers', '7': 'moon orchid', '16': 'globe-flower', '25': 'grape hyacinth', '26': 'corn poppy', '79': 'toad lily', '39': 'siam tulip', '24': 'red ginger', '67': 'spring crocus', '35': 'alpine sea holly', '32': 'garden phlox', '10': 'globe thistle', '6': 'tiger lily', '93': 'ball moss', '33': 'love in the mist', '9': 'monkshood', '102': 'blackberry lily', '14': 'spear thistle', '19': 'balloon flower', '100': 'blanket flower', '13': 'king protea', '49': 'oxeye daisy', '15': 'yellow iris', '61': 'cautleya spicata', '31': 'carnation', '64': 'silverbush', '68': 'bearded iris', '63': 'black-eyed susan', '69': 'windflower', '62': 'japanese anemone', '20': 'giant white arum lily', '38': 'great masterwort', '4': 'sweet pea', '86': 'tree mallow', '101': 'trumpet creeper', '42': 'daffodil', '22': 'pincushion flower', '2': 'hard-leaved pocket orchid', '54'

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

In [9]:
# Use torchvision to import pretrained model
model = models.vgg16(pretrained=True)
  
# print model summary
print(str(model))

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.torch/models/vgg16-397923af.pth
100%|██████████| 553433881/553433881 [00:07<00:00, 74806051.88it/s]


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

In [10]:
# TODO
# Turn off training for the parameters
for param in model.parameters():
    param.requires_grad = False

In [11]:
# Define a new, untrained feed-forward network 
# as a classifier, using ReLU activations and dropout
#  TODO: DO I NEED THESE?
#classifier_input = model.classifier.in_features
num_of_labels = 102 # number of flower names/labels

# TODO: set up new classifier

# TODO   is the out_features set to number of classes 102?
# play with numbers to start at 25088 and end at 102
# TODO do we need (OrderedDict after sequential?

# TODO: should dim = 1 (rows) or 0 (columns)

classifier = nn.Sequential(nn.Linear(25088, 500),
                           nn.ReLU(),
                           nn.Dropout(0.5),
                           nn.Linear(500, 102),
                           #nn.ReLU(),
                           #nn.Dropout(0.05),
                           #nn.Linear(1000, num_of_labels),
                           #nn.ReLU(),
                           #nn.Dropout(0.05),
                           nn.LogSoftmax(dim=1)
                           )
                           
                           
#     ('fc1',nn.Linear(25088, 12000)),
#     ('relu1',nn.ReLU()),
#     ('dropout1',nn.Dropout(0.5)),
#     ('fc2',nn.Linear(12000,102)),
#     #('relu1',nn.ReLU()),
#     #('dropout1',nn.Dropout(0.5)),
#     #('fc3',nn.Linear(3060,102))
#     #('output', nn.LogSoftmax(dim=1)),
#     ])
    
model.classifier = classifier


In [12]:
print(classifier)

Sequential(
  (0): Linear(in_features=25088, out_features=500, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.5)
  (3): Linear(in_features=500, out_features=102, bias=True)
  (4): LogSoftmax()
)


In [14]:
# make a loop to flatten images?
for images in train_loaders:
    images = images.view(images.shape[0], -1)
        # Flatten image     images = images.view(images.shape[0], -1)
        
        # Get a batch of training data
#inputs, classes = next(iter(dataloaders['train']))

AttributeError: 'list' object has no attribute 'view'

In [72]:
for param in model.parameters():
    print(param)

Parameter containing:
tensor([[[[-0.5537,  0.1427,  0.5290],
          [-0.5831,  0.3566,  0.7657],
          [-0.6902, -0.0480,  0.4841]],

         [[ 0.1755,  0.0099, -0.0814],
          [ 0.0441, -0.0703, -0.2604],
          [ 0.1324, -0.1728, -0.1323]],

         [[ 0.3130, -0.1659, -0.4275],
          [ 0.4752, -0.0827, -0.4870],
          [ 0.6320,  0.0193, -0.2775]]],


        [[[ 0.2325,  0.1267,  0.1861],
          [-0.4281, -0.2435,  0.2463],
          [-0.2507,  0.1418, -0.0055]],

         [[-0.1408, -0.2190,  0.1504],
          [-0.8413, -0.3518,  0.5640],
          [-0.2419,  0.5193,  0.5391]],

         [[-0.3143, -0.3705, -0.1309],
          [-0.4714, -0.1550,  0.3459],
          [ 0.0544,  0.5868,  0.4958]]],


        [[[ 0.1772,  0.5215,  0.0099],
          [-0.2718, -0.7171,  0.3129],
          [-0.0758, -0.2208,  0.3346]],

         [[ 0.3092,  0.6707,  0.0205],
          [-0.4661, -1.0697,  0.3350],
          [-0.0803, -0.3052,  0.5446]],

         [[ 0.3157,  0

In [102]:
# per video
criterion = nn.NLLLoss()

# parameters are what we want the model to update


# lr is learning rate
learning_rate = 0.003

# parameters update every time - refer to the parameters
# of the new classifier

# optimizer = optim.SGD(model.classifier.parameters(), lr=learning_rate )
optimizer = optim.SGD(lr=learning_rate)

epochs = 5
for e in range(epochs):
    running_loss = 0
    for images, labels in train_loaders:
        # Flatten image
        images = images.view(images.shape[0], -1)
        
        # TODO: trainging pass
        optimizer.zero_grad()
        
        output = model.classifier.forward(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    else:
        print(f"Training loss: {running_loss/len(train_loaders)}")

TypeError: __init__() missing 1 required positional argument: 'params'

In [74]:
# Train the classifier layers using backpropagation using the pre-trained network to get the features





In [22]:
# From network architectures solution video

# Track the loss and accuracy on the validation set to determine the best hyperparameters

# TODO... from video

# Define the loss using 
criterion = nn.NLLLoss()


# TODO not sure where this goes, 
# Not sure if it should be SGA or ADAM
# parameters are what we want the model to update
# lr is learning rate
optimizer = optim.SGA(model.parameters(), lr=0.01)

# Get the data
images, labels = next(iter(train_loaders))

# Flatten images to 102 labels
images = images.view(images.shape[0], -1)

# Forward pass, get our logits
logps = model(images)

# Calculate the loss with the logits and the labels
loss = criterion(logps, labels)

print(loss)

print('Before backward pass: \n', model[0].weight.grad)

loss.backward()

print('After backward pass: \n', model



# Option 2
# Define the loss using CrossEntropyLoss
# criterion = nn.CrossEntropyLoss()

# TODO  is this for each one train, valid, test?
# Get our data 
# images, labels = next(iter(train_loaders))

# Flatten images in 102 assigned labels
# images = images.view(images.shape[0], -1)

# Forward pass, get our logits
#logits = model(images)

# Calculate the loss with the logits and the labels
# loss = criterion(logits, labels)

# print(loss)

RuntimeError: input has less dimensions than expected

In [22]:
# From network architectures solution video

# Track the loss and accuracy on the validation set to determine the best hyperparameters

# TODO... from video

# Define the loss using 
criterion = nn.NLLLoss()

# Get the data
images, labels = next(iter(train_loaders))

# Flatten images to 102 labels
images = images.view(images.shape[0], -1)

# Forward pass, get our logits
logps = model(images)

# Calculate the loss with the logits and the labels
loss = criterion(logps, labels)

print(loss)

print('Before backward pass: \n', model[0].weight.grad)

loss.backward()

print('After backward pass: \n', model[0].weight.grad)



# Option 2
# Define the loss using CrossEntropyLoss
# criterion = nn.CrossEntropyLoss()

# TODO  is this for each one train, valid, test?
# Get our data 
# images, labels = next(iter(train_loaders))

# Flatten images in 102 assigned labels
# images = images.view(images.shape[0], -1)

# Forward pass, get our logits
#logits = model(images)

# Calculate the loss with the logits and the labels
# loss = criterion(logits, labels)

# print(loss)

RuntimeError: input has less dimensions than expected

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [3]:
# TODO: Do validation on the test set




## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [2]:
# TODO: Save the checkpoint 
# exam torch.save(model.state_dict(), 'checkpoint.pth')

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
# TODO: Write a function that loads a checkpoint and rebuilds the model
# exp  print(state_dict.keys())

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    
    # TODO: Process a PIL image for use in a PyTorch model

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # TODO: Implement the code to predict the class from an image file

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# TODO: Display an image along with the top 5 classes

In [ ]:
# tips:
# To see the directory allocated to you, click the "Jupyter notebook" logo > aipnd-project > flowers.
# You will be able to see the test, train, valid folders there

# to print out list of attributes of model example
# print(str(vgg16)) 

# remember to note the path when trying to run program
# python ImageClassifier/train.py